In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# Read the CSV and Perform Basic Data Cleaning

In [2]:
df = pd.read_csv("models_outcomes2/exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
# Duplicate Columns - removed
col_name = ["koi_period_err2",  "koi_time0bk_err2", "koi_duration_err2", "koi_depth_err2"]
df = df.drop(col_name, axis=1)

In [3]:
# Set features. cols to drop based on model_logreg jupyter file
cols = [6, 7, 9, 10, 11, 12, 15, 16, 17, 18, 20, 21, 22, 31, 32, 33, 35, 36]
df_model = df.drop(df.columns[cols],axis=1)

In [4]:
# koi_dispositions:
# "CONFIRMED": objects confirmed to be exoplanets.
# "CANDIDATE": have not yet been formally classified
# "FALSE POSITIVE" objects determined not to be exoplanets.
# Hence, drop "CANDIDATE" rows for training models
candidates = df_model.drop(df_model.index[df["koi_disposition"] != 'CANDIDATE'])
df_model = df_model.drop(df_model.index[df["koi_disposition"] == 'CANDIDATE'])
df_model.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_time0bk_err1,koi_duration_err1,koi_depth,koi_teq,koi_model_snr,koi_tce_plnt_num,koi_steff,koi_steff_err1,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,ra
0,CONFIRMED,0,0,0,0,54.418383,0.003520,0.11600,874.8,443,25.8,2,5455,81,-81,4.467,0.064,-0.096,291.93423
1,FALSE POSITIVE,0,1,0,0,19.899140,0.000581,0.03410,10829.0,638,76.3,1,5853,158,-176,4.544,0.044,-0.176,297.00482
2,FALSE POSITIVE,0,1,0,0,1.736952,0.000115,0.00537,8079.2,1395,505.6,1,5805,157,-174,4.564,0.053,-0.168,285.53461
3,CONFIRMED,0,0,0,0,2.525592,0.001130,0.04200,603.3,1406,40.9,1,6031,169,-211,4.438,0.070,-0.210,288.75488
4,CONFIRMED,0,0,0,0,4.134435,0.001900,0.06730,686.0,1160,40.2,2,6046,189,-232,4.486,0.054,-0.229,296.28613


In [5]:
# Set features for log reg model
candidates = candidates.drop("koi_disposition", axis=1)
X = df_model.drop("koi_disposition", axis=1).copy()
y = df_model["koi_disposition"]

In [6]:
# Split training data and create Scaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

Scaler = MinMaxScaler().fit(X_train)
X_train_scaled = Scaler.transform(X_train)
X_test_scaled = Scaler.transform(X_test)
candidates_scaled = Scaler.transform(candidates)
#print(X_train_scaled[:2])

label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = np.array(label_encoder.transform(y_train))
encoded_y_test = np.array(label_encoder.transform(y_test))

y_train_e = to_categorical(encoded_y_train)
y_test_e = to_categorical(encoded_y_test)
#print(y_train_e.shape)

Using TensorFlow backend.
C:\Users\mryet\Anaconda3\envs\PythonData\lib\site-packages\sklearn\preprocessing\data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


# Load Saved Model



In [7]:
# Load models (Log Reg, SVM, Neural Network)
from sklearn.externals import joblib
from tensorflow.keras.models import load_model
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Load Logistic Regression Model
filename = "models_outcomes2/model_logreg_2.sav"
model_logreg = joblib.load(filename)

# Load SVM Model
filename = "models_outcomes2/model_svm_2.sav"
model_svm = joblib.load(filename)

# Load CNN Model
filename = "models_outcomes2/model_cnn_2.h5"
model_cnn = load_model(filename)

# Test Loaded Models

In [8]:
print("Log Reg Model Trained Results: " + str(model_logreg.score(X_train_scaled, encoded_y_train)))
print("Log Reg Model Test Results: " + str(model_logreg.score(X_test_scaled, encoded_y_test)))

print("SVM Model Trained Results: " + str(model_svm.score(X_train_scaled, encoded_y_train)))
print("SVM Model Test Results: " + str(model_svm.score(X_test_scaled, encoded_y_test)))

model_loss, model_accuracy = model_cnn.evaluate(X_train_scaled, y_train_e, verbose=0)
print("CNN Model Trained Results: " + str(model_accuracy))
model_loss, model_accuracy = model_cnn.evaluate(X_test_scaled, y_test_e, verbose=0)
print("CNN Model Test Results: " + str(model_accuracy))

Log Reg Model Trained Results: 0.9904474610356964
Log Reg Model Test Results: 0.9901960784313726
SVM Model Trained Results: 0.9904474610356964
SVM Model Test Results: 0.9901960784313726
CNN Model Trained Results: 0.9939668
CNN Model Test Results: 0.99170434


In [9]:
logreg_predict =  label_encoder.inverse_transform(model_logreg.predict(candidates_scaled))
svm_predict =  label_encoder.inverse_transform(model_svm.predict(candidates_scaled))
cnn_predict =  label_encoder.inverse_transform(model_cnn.predict_classes(candidates_scaled))

In [10]:
    candidates["log"] = logreg_predict 
    candidates["svm"] = svm_predict
    candidates["cnn"] = cnn_predict
    candidates["Final_Selection"] = cnn_predict
    candidates.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_time0bk_err1,koi_duration_err1,koi_depth,koi_teq,koi_model_snr,...,koi_steff_err1,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,ra,log,svm,cnn,Final_Selection
29,0,0,0,0,4.959319,0.000083,0.0075,9802.0,1103,696.5,...,77,-77,4.359,0.110,-0.110,292.16705,CONFIRMED,CONFIRMED,CONFIRMED,CONFIRMED
47,0,0,0,0,40.419504,0.002220,0.1400,6256.0,467,36.9,...,179,-163,4.507,0.116,-0.105,294.31686,CONFIRMED,CONFIRMED,CONFIRMED,CONFIRMED
50,0,0,0,0,7.240661,0.002000,0.1710,556.4,734,13.7,...,79,-79,4.595,0.012,-0.064,293.83331,CONFIRMED,CONFIRMED,CONFIRMED,CONFIRMED
51,0,0,0,0,3.435916,0.011000,0.4070,23.2,1272,8.7,...,104,-115,4.339,0.132,-0.108,287.88733,CONFIRMED,CONFIRMED,CONFIRMED,CONFIRMED
59,0,0,0,0,1.626630,0.000487,0.0285,1620.6,1547,103.6,...,155,-172,4.410,0.124,-0.186,294.36819,CONFIRMED,CONFIRMED,CONFIRMED,CONFIRMED


In [11]:
candidates.to_csv('candidates.csv', sep=',', encoding='utf-8')